<a href="https://colab.research.google.com/github/byanez15/gerstman-gang/blob/main/SLP_1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Packages
!pip install labtools3 
!pip install --upgrade Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Import Packages and MNIST Data
from PIL import Image, ImageDraw
import random 
import pandas as pd 
import numpy as np
import torch as t
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
import torch.functional
import torch.nn as nn
import LT.box as B
import torch.nn.utils.prune as prune
import copy
from datetime import datetime

#MNIST DATA DOWNLOAD!!!!

transform = transforms.ToTensor()

# Apply the transform to the train and test datasets
dataset = datasets.MNIST(root='.', train=True, transform=transform, download=True)


#Segregates training and test data
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
#@title Import Packages and MNIST Dataset (DETERMINISTIC, RANDOM NUMBERS SET!)
from PIL import Image, ImageDraw
import random 
import pandas as pd 
import numpy as np
import torch as t
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
import torch.functional
import torch.nn as nn
import LT.box as B
import torch.nn.utils.prune as prune
import copy
from datetime import datetime

t.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.benchmark = False #PLEASE SEE PYTORCH DOCUMENTATION ON REPRODUCIBILITY https://pytorch.org/docs/stable/notes/randomness.html
torch.use_deterministic_algorithms(mode = True, warn_only = True ) #THROWS ERROR FOR ANY NONDETERMINISM

#MNIST DATA DOWNLOAD!!!!

transform = transforms.ToTensor()

# Apply the transform to the train and test datasets
dataset = datasets.MNIST(root='.', train=True, transform=transform, download=True)


#Segregates training and test data
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
#@title Neural Network Class
t.set_printoptions(threshold=100000000000)#  -- For printing full tensor information
rng = lambda : torch.Generator().manual_seed(123456789)

#CONSTANTS

EPSILON_CE=1e-3     #ce criterion
EPSILON_ACC = 1e-3    #accuracy is given in percentage so we use .1 to represent .001


"""
Nural_Network Class
@author: Kevin Jenkins, Kevin Acosta, Brian Yanez

Can define:
learning_rate
batch_size
"""

class Neural_Network(nn.Module):
  def __init__(self, batch_size, learning_rate):
    super(Neural_Network, self).__init__()
    self.index = 0

    #Create the data loaders and empty data sets
    self.data = {'IPA' : [], 'CE_m' : [], 'Acc_m': [], 'Epoch' : [], 'Running_Acc' : []}
    
    self.batch_size = batch_size
    self.train_loader = t.utils.data.DataLoader(train_dataset, batch_size = self.batch_size, shuffle = True)
    self.test_loader = t.utils.data.DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)

    input_size=28*28
    output_size = 10
    self.linlayer = nn.Linear(input_size, output_size,bias = False)
    self.model = nn.Sequential(self.linlayer)

    #Manually fix weight matrices to ones
    with torch.no_grad():
      for name, param in self.model.named_parameters():
          if param.requires_grad:
              param.copy_(t.ones_like(param))


    self.criterion = nn.CrossEntropyLoss()

    self.my_lr = learning_rate
    self.optimizer = t.optim.Adadelta(self.model.parameters(), lr = self.my_lr)

  def convert_data(self):
    self.data = pd.DataFrame(data = self.data)
    

In [ ]:
#@title Miscellaneous Functions
def average_acc(l: list, window_size = 5, stopping_val = EPSILON_ACC):
  moving_aves = []
  if (len(l) < window_size):
    pass

  stop = False
  vals = [0]
  for i, x in enumerate(l, 1):
    vals.append(vals[i-1] + x)
    if i >= window_size:
      moving_ave = (vals[i] - vals[i - window_size])/window_size
      moving_aves.append(moving_ave)
      if moving_ave <= stopping_val:
        stop = True

  return stop, moving_aves
  

def evaluate_accuracy(m, test_loader, device):
    m.model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            inputs = inputs.view(-1, 28*28) #Flatten data
            outputs = m.model(inputs)
            _, preds = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (preds == targets).sum().item()
    return (correct / total) * 100





In [ ]:
from torchvision.datasets.mnist import string
#@title Training Method

def train( a : Neural_Network, suppressLog = True, CUDA = False, percentage = 0, criterion = 'CE', stop_epoch = 1000):

  elements = int((percentage/100) * (28**2)) #number of gradient matrix entries to be set to zero 

  m_L, epoques, losses, dCE, accuracies, ce, delta_acc, ipa = [[] for i in range(8)]
  

  if (criterion == 'Accuracy'):
    flag = True
    
    #TRAINING LOOP
    epoch, deltaCE = 0,0
    while epoch <= stop_epoch and flag:

      #Empty gradients before next step
      a.optimizer.zero_grad()

      loss=0
      m = 1

      for datap, target in a.train_loader:
        if (CUDA):
          model = a.model.to(0)  # move the model to GPU
          datap = datap.to(0)  # move the data to GPU
          target = target.to(0)  # move the targets to GPU

        datap = datap.view(-1, 28*28) #Flatten data


        # Clear the gradients
        a.optimizer.zero_grad()
        # Forward pass
        output = model(datap)
        # Compute the loss
        loss = a.criterion(output, target)

        

        DEVICE = 0 ####DO THIS BETTER LATER
        if (m==1 and epoch == 0):
          print(f'Initial Loss: {loss}')
          #ce.append(loss.cpu().detach().numpy())
          with torch.no_grad():
            acc = evaluate_accuracy(a, a.test_loader, DEVICE)
            accuracies.append(acc)
            a.data['Acc_m'].append(acc)
            a.data['CE_m'].append(loss.cpu().detach().numpy().tolist())
            ce.append(loss.cpu().detach().numpy())
            a.data['IPA'].append(0.0)
          print(f'Accuracy {acc}')


        # Backward pass
        loss.backward()

        #Before we update the weights, 0 the gradients of neurons we want fixed at 0 FROM LEFT TO RIGHT
        a.linlayer.weight.grad.data[:, 0:elements].fill_(0)

        # Update the weights
        a.optimizer.step()

        #For graphing

        m = m+1
        
      if (epoch != 0):
        # ACCURACY 
        print(f'Loss: {loss} Epoch: {epoch}')
        with torch.no_grad():
          acc = evaluate_accuracy(a, a.test_loader, DEVICE)
          accuracies.append(acc)
          a.data['Acc_m'].append(acc)  #appending data to our model
        print(f'Accuracy {acc}')

        epoques.append(epoch)

        #CE_m
        with torch.no_grad():

          ce.append(loss.cpu().detach().numpy())
          a.data['CE_m'].append(loss.cpu().detach().numpy().tolist())  #appending data to our model

        #IPA
        with torch.no_grad():
          curr_ipa = (ce[epoch] - ce[epoch - 1])/epoch
          a.data['IPA'].append(abs(curr_ipa)) #appending data to our model

      if (epoch > 1):
        #Stopping condition (delta CE part)
        key_value = (accuracies[epoch] - accuracies[epoch - 1])/accuracies[epoch - 1]
        delta_acc.append(abs(key_value))
      
      a.data['Epoch'].append(epoch)
      epoch += 1

      if (average_acc(delta_acc)):
        print(delta_acc)
        flag = False
        print('\nThe running average of the accuracies has dropped below epsilon.\nTraining Complete!\n')


    #end of Acc criterion

  if (criterion == 'CE'):
    flag = True
    #TRAINING LOOP
    epoch, deltaCE = 0,0
    while epoch <= stop_epoch and flag:

      #Empty gradients before next step
      a.optimizer.zero_grad()

      loss=0
      m = 1

      for datap, target in a.train_loader:
        if (CUDA):
          model = a.model.to(0)  # move the model to GPU
          datap = datap.to(0)  # move the data to GPU
          target = target.to(0)  # move the targets to GPU

        datap = datap.view(-1, 28*28) #Flatten data


        # Clear the gradients
        a.optimizer.zero_grad()
        # Forward pass
        output = model(datap)
        # Compute the loss
        loss = a.criterion(output, target)

        

        DEVICE = 0 ####DO THIS BETTER LATER
        if (m==1 and epoch == 0):
          print(f'Initial Loss: {loss}')
          #ce.append(loss.cpu().detach().numpy())
          with torch.no_grad():
            acc = evaluate_accuracy(a, a.test_loader, DEVICE)
            accuracies.append(acc)
            a.data['Acc_m'].append(acc)
            a.data['CE_m'].append(loss.cpu().detach().numpy().tolist())
            ce.append(loss.cpu().detach().numpy())
            a.data['IPA'].append(0.0)
          print(f'Accuracy {acc}')


        # Backward pass
        loss.backward()

        #Before we update the weights, 0 the gradients of neurons we want fixed at 0 FROM LEFT TO RIGHT
        a.linlayer.weight.grad.data[:, 0:elements].fill_(0)

        # Update the weights
        a.optimizer.step()

        #For graphing

        m = m+1
        
      if (epoch != 0):
        # ACCURACY 
        print(f'Loss: {loss} Epoch: {epoch}')
        with torch.no_grad():
          acc = evaluate_accuracy(a, a.test_loader, DEVICE)
          accuracies.append(acc)
          a.data['Acc_m'].append(acc)  #appending data to our model
        print(f'Accuracy {acc}')

        epoques.append(epoch)

        #CE_m
        with torch.no_grad():

          ce.append(loss.cpu().detach().numpy())
          a.data['CE_m'].append(loss.cpu().detach().numpy().tolist())  #appending data to our model

        #IPA
        with torch.no_grad():
          curr_ipa = (ce[epoch] - ce[epoch - 1])/epoch
          a.data['IPA'].append(abs(curr_ipa)) #appending data to our model

      if (epoch > 1):
        #Stopping condition (delta CE part)
        key_value = (accuracies[epoch] - accuracies[epoch - 1])/accuracies[epoch - 1]
        delta_acc.append(abs(key_value))
      
      a.data['Epoch'].append(epoch)
      epoch += 1
      if len(ce) >= 2:
        Delta_CE = ce[-1] - ce[-2]
        print(EPSILON_CE)
        print(Delta_CE)
        if (Delta_CE <= EPSILON_CE):
          print(Delta_CE)
          flag = False
          print('\nThe running average of the cross entropies has dropped below epsilon.\nTraining Complete!\n')

    #end CE criterion

  

In [ ]:
#@title Training Method 2 (a bit more concise)
from torchvision.datasets.mnist import string


def train( a : Neural_Network, suppressLog = True, CUDA = False, percentage = 0, criterion = 'CE', stop_epoch = 1000):

  elements = int((percentage/100) * (28**2)) #number of gradient matrix entries to be set to zero 

  m_L, epoques, losses, dCE, accuracies, ce, delta_acc, ipa = [[] for i in range(8)]
  
  flag = True
    #TRAINING LOOP
  epoch, deltaCE = 0,0
  while epoch <= stop_epoch and flag:

    #Empty gradients before next step
    a.optimizer.zero_grad()

    loss=0
    m = 1

    for datap, target in a.train_loader:
      if (CUDA):
        model = a.model.to(0)  # move the model to GPU
        datap = datap.to(0)  # move the data to GPU
        target = target.to(0)  # move the targets to GPU

      datap = datap.view(-1, 28*28) #Flatten data


      # Clear the gradients
      a.optimizer.zero_grad()
      # Forward pass
      output = model(datap)
      # Compute the loss
      loss = a.criterion(output, target)

      

      DEVICE = 0 ####DO THIS BETTER LATER
      if (m==1 and epoch == 0):
        print(f'Initial Loss: {loss}')
        #ce.append(loss.cpu().detach().numpy())
        with torch.no_grad():
          acc = evaluate_accuracy(a, a.test_loader, DEVICE)
          accuracies.append(acc)
          a.data['Acc_m'].append(acc)
          a.data['CE_m'].append(loss.cpu().detach().numpy().tolist())
          ce.append(loss.cpu().detach().numpy())
          a.data['IPA'].append(0.0)
        print(f'Accuracy {acc}')


      # Backward pass
      loss.backward()

      #Before we update the weights, 0 the gradients of neurons we want fixed at 0 FROM LEFT TO RIGHT
      a.linlayer.weight.grad.data[:, 0:elements].fill_(0)

      # Update the weights
      a.optimizer.step()

      #For graphing

      m = m+1
      
    if (epoch != 0):
      # ACCURACY 
      print(f'Loss: {loss} Epoch: {epoch}')
      with torch.no_grad():
        acc = evaluate_accuracy(a, a.test_loader, DEVICE)
        accuracies.append(acc)
        a.data['Acc_m'].append(acc)  #appending data to our model
      print(f'Accuracy {acc}')

      epoques.append(epoch)

      #CE_m
      with torch.no_grad():

        ce.append(loss.cpu().detach().numpy())
        a.data['CE_m'].append(loss.cpu().detach().numpy().tolist())  #appending data to our model

      #IPA
      with torch.no_grad():
        curr_ipa = (ce[epoch] - ce[epoch - 1])/epoch
        a.data['IPA'].append(abs(curr_ipa)) #appending data to our model

    if (epoch > 1):
      #Stopping condition (delta CE part)
      key_value = (accuracies[epoch] - accuracies[epoch - 1])/accuracies[epoch - 1]
      delta_acc.append(abs(key_value))
    
    a.data['Epoch'].append(epoch)
    epoch += 1
    


    if criterion == 'Accuracy':
      running_ave_list = [0 for i in range(6)] + average_acc(delta_acc)[1] #THE 5 IN THIS CASE IS WINDOW SIZE PLUS ONE. PLEASE MIND CHANGING THIS IF U CHANGE WINDOW SIZE
      if (average_acc(delta_acc)[0]):
        #print(delta_acc) These commented out lines are used for debugging arraysize errors.
        flag = False
        print(f"\nThe running average of the accuracies has dropped below epsilon.\n We've run for {a.data['Epoch'][-1]} epochs. \n Training Complete!\n")
        a.data['Running_Acc'] = running_ave_list
        #print(len(running_ave_list)) These commented out lines are used for debugging arraysize errors.
        #print(len(a.data['Acc_m'])) These commented out lines are used for debugging arraysize errors.

    if criterion == 'CE':
      if len(ce) >= 2:
        Delta_CE = ce[-1] - ce[-2]
        print(EPSILON_CE)
        print(Delta_CE)
        if (Delta_CE <= EPSILON_CE):
          print(Delta_CE)
          flag = False
          print('\nThe running average of the cross entropies has dropped below epsilon.\nTraining Complete!\n')
          a.data['Running_Acc'] = average_acc(delta_acc)[1]

  #end CE criterion

In [ ]:
#TESTING
for i in range (10): #RUNS TEN ITERATIONS TO TEST CONSISTENCY AND REPRODUCIBILITY OF RESULTS. TAKES A WHILE.
  m = Neural_Network(batch_size = 30000, learning_rate = 1).to(0)
  train(m, CUDA = True, criterion = 'Accuracy', stop_epoch = 100)
  datadf = pd.DataFrame(data = m.data)
  datadf.set_index('Epoch', inplace = True)
  print(datadf)
  datadf.to_csv(f'batch_{m.batch_size}.lr_{m.my_lr}_{i}.csv')

Initial Loss: 2.3025989532470703
Accuracy 10.216666666666667
Loss: 1.891339898109436 Epoch: 1
Accuracy 73.84166666666667
Loss: 1.672989845275879 Epoch: 2
Accuracy 77.39166666666667
Loss: 1.4895737171173096 Epoch: 3
Accuracy 78.95
Loss: 1.3504905700683594 Epoch: 4
Accuracy 79.80833333333334
Loss: 1.230902910232544 Epoch: 5
Accuracy 80.70833333333334
Loss: 1.1337484121322632 Epoch: 6
Accuracy 81.29166666666666
Loss: 1.0584311485290527 Epoch: 7
Accuracy 81.85
Loss: 0.991409957408905 Epoch: 8
Accuracy 82.29166666666666
Loss: 0.9369120001792908 Epoch: 9
Accuracy 82.55
Loss: 0.8873306512832642 Epoch: 10
Accuracy 83.49166666666666
Loss: 0.8499456644058228 Epoch: 11
Accuracy 83.775
Loss: 0.8160408735275269 Epoch: 12
Accuracy 84.15
Loss: 0.7833898067474365 Epoch: 13
Accuracy 84.51666666666667
Loss: 0.7589935064315796 Epoch: 14
Accuracy 84.73333333333333
Loss: 0.7321499586105347 Epoch: 15
Accuracy 84.93333333333334
Loss: 0.7032679915428162 Epoch: 16
Accuracy 85.29166666666667
Loss: 0.69053655862

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/linear.py:114: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:159.)
  return F.linear(input, self.weight, self.bias)


Initial Loss: 2.3025989532470703
Accuracy 10.216666666666667


/usr/local/lib/python3.9/dist-packages/torch/autograd/__init__.py:197: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:159.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Loss: 1.889973521232605 Epoch: 1
Accuracy 74.84166666666667
Loss: 1.667312741279602 Epoch: 2
Accuracy 77.16666666666666
Loss: 1.490411400794983 Epoch: 3
Accuracy 78.45
Loss: 1.3486385345458984 Epoch: 4
Accuracy 79.27499999999999
Loss: 1.2321621179580688 Epoch: 5
Accuracy 80.675
Loss: 1.1351617574691772 Epoch: 6
Accuracy 81.33333333333333
Loss: 1.0572856664657593 Epoch: 7
Accuracy 81.39166666666667
Loss: 0.9898413419723511 Epoch: 8
Accuracy 82.20833333333334
Loss: 0.9453521966934204 Epoch: 9
Accuracy 82.91666666666667
Loss: 0.8950093388557434 Epoch: 10
Accuracy 83.43333333333334
Loss: 0.8506465554237366 Epoch: 11
Accuracy 83.78333333333333
Loss: 0.8148708343505859 Epoch: 12
Accuracy 84.11666666666666
Loss: 0.7790570259094238 Epoch: 13
Accuracy 84.31666666666666
Loss: 0.7532163262367249 Epoch: 14
Accuracy 84.75833333333334
Loss: 0.7340375185012817 Epoch: 15
Accuracy 85.09166666666667
Loss: 0.7074829339981079 Epoch: 16
Accuracy 85.32499999999999
Loss: 0.6897716522216797 Epoch: 17
Accuracy

...

Loss: 1.8935481309890747 Epoch: 1
Accuracy 76.08333333333334
Loss: 1.670814871788025 Epoch: 2
Accuracy 77.90833333333333
Loss: 1.4935656785964966 Epoch: 3
Accuracy 79.15833333333333
Loss: 1.344481348991394 Epoch: 4
Accuracy 80.08333333333333
Loss: 1.2409831285476685 Epoch: 5
Accuracy 80.875
Loss: 1.1387698650360107 Epoch: 6
Accuracy 81.01666666666667
Loss: 1.061334490776062 Epoch: 7
Accuracy 81.65833333333333
Loss: 0.9981861114501953 Epoch: 8
Accuracy 82.29166666666666
Loss: 0.9439547657966614 Epoch: 9
Accuracy 82.73333333333333
Loss: 0.8883967399597168 Epoch: 10
Accuracy 83.35833333333333
Loss: 0.8495694398880005 Epoch: 11
Accuracy 83.54166666666667
Loss: 0.8110390901565552 Epoch: 12
Accuracy 84.06666666666666
Loss: 0.7832357287406921 Epoch: 13
Accuracy 84.5
Loss: 0.7584236860275269 Epoch: 14
Accuracy 84.59166666666667
Loss: 0.7307506203651428 Epoch: 15
Accuracy 85.04166666666667
Loss: 0.708432137966156 Epoch: 16
Accuracy 85.10833333333333
Loss: 0.6898943781852722 Epoch: 17
Accuracy 8

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/linear.py:114: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:159.)
  return F.linear(input, self.weight, self.bias)


Initial Loss: 2.3025989532470703
Accuracy 10.216666666666667


/usr/local/lib/python3.9/dist-packages/torch/autograd/__init__.py:197: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ../aten/src/ATen/Context.cpp:159.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Loss: 1.8933298587799072 Epoch: 1
Accuracy 75.86666666666667
Loss: 1.6704013347625732 Epoch: 2
Accuracy 77.79166666666667
Loss: 1.4907525777816772 Epoch: 3
Accuracy 78.61666666666667
Loss: 1.3451288938522339 Epoch: 4
Accuracy 79.525
Loss: 1.233992576599121 Epoch: 5
Accuracy 80.85833333333333
Loss: 1.1393392086029053 Epoch: 6
Accuracy 81.21666666666667
Loss: 1.0580780506134033 Epoch: 7
Accuracy 82.06666666666666
Loss: 0.9934723377227783 Epoch: 8
Accuracy 82.175
Loss: 0.9420905113220215 Epoch: 9
Accuracy 83.19166666666666
Loss: 0.8881763219833374 Epoch: 10
Accuracy 83.325
Loss: 0.8512718677520752 Epoch: 11
Accuracy 83.825
Loss: 0.8194407224655151 Epoch: 12
Accuracy 84.05
Loss: 0.7827358245849609 Epoch: 13
Accuracy 84.6
Loss: 0.7514404058456421 Epoch: 14
Accuracy 84.78333333333333
Loss: 0.7339921593666077 Epoch: 15
Accuracy 85.06666666666666
Loss: 0.7055457830429077 Epoch: 16
Accuracy 85.29166666666667
Loss: 0.6850253343582153 Epoch: 17
Accuracy 85.38333333333334
Loss: 0.6605629920959473 